Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [15]:
from random import choices, randint, choice, sample, random
from copy import copy, deepcopy

import lab9_lib
import numpy as np
import math


In [16]:
fitness = lab9_lib.make_problem(1)
for n in range(10):     #Determina il numero di Fitness call
    ind = choices([0, 1], k=1000)     #genoma casuale di 50 elementi
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

1110110100100011101000000110011111110011001000010001010001000110011101111110110100011110000010011101001000011100101111110101111010101000100110001100011001011101010000010100000110000111010101001011000110111010101110010000111110111111010000000000101000000110000101011100101101010000000101010011011001101100010011101010100000100101010101110001010101000001001100101100001011110010000101101000011000101010110000011010111000011001010010011010101110111000100101000100010010110111100000111100111011110011110001000011011110111100011101011001010011011101101010100101001100011101100101011100001100011100000101000111111000100100111111011001010010010101101010101000110100000010111101000110111011101010000100110010110010011101110111100111011111101001000000001111001110111100011001001010111100110100100001010000000001011000110001000010001100110110110100011111111100101110101011010010000010111101001100011111010110110111110011101000111100011000011111111101000001110000101010010111001010001100100001000000000100011101

# Local Search Algorithm

In [17]:
# GLOBAL PARAMETER #
NUM_LOCI = 1000

POPULATION_SIZE = 100
NEW_OFFSPRING = 30
TOURNAMENT_SIZE = 5
MUTATION_PROBABILITY = .15
NUM_ISLANDS = 3

# . . . . . . #

### Mutation Techniques

In [18]:
def mutation1(ind):     # 1 loci(gene) mutated
    offspring = copy(ind)
    pos = randint(0, NUM_LOCI-1)
    offspring[pos] = 1 - offspring[pos]     # Se ho T/F-> offspring[pos] = not offspring[pos]
    #assert len(offspring) == NUM_LOCI
    return offspring

def mutation2(ind):     # Reset randomly a random number of loci
    offspring = copy(ind)
    poss = (randint(0, NUM_LOCI-1), randint(0, NUM_LOCI-1))
    offspring = ind[:min(poss)] + [choice([0, 1])for _ in range(max(poss)-min(poss))] + ind[max(poss):]
    #assert len(offspring) == NUM_LOCI

    return offspring

### Recombination Techniques

In [19]:
def one_cut_crossover(ind1, ind2):
    cut_point = randint(0, NUM_LOCI-1)
    offspring = ind1[:cut_point] + ind2[cut_point:]
    #    assert len(offspring) == NUM_LOCI
    return offspring

'''def n_cut_crossover(ind1: fitness, ind2: fitness) -> fitness:
    num_cut = math.ceil(np.random.normal(loc=10, scale=5))
    cut_point = randint(0, NUM_LOCI-1)
    offspring = ind1[:(NUM_LOCI/num_cut)]

    offspring = ind1[:cut_point] + ind2[cut_point:]
    assert len(offspring.genotype) == NUM_LOCI
    return offspring'''

def uniform_crossover(ind1, ind2):
    offspring = [ind1[i] if i % 2 else ind2[i] for i in range(NUM_LOCI)]
    # assert len(offspring) == NUM_LOCI
    return offspring

### Parent Selection

In [20]:
def roulette_wheel(population):     # Roulette wheel with same probability
    return choice(population)

def variable_tournament(population):
    
    n_partecipants = randint(0, int(len(population)/2))
    pool = [choices(population, k=n_partecipants)]
    parent = max(pool, key=lambda i: fitness(i))

    return parent

def static_tournament(pop):

    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champ = max(pool, key=lambda i: fitness(i))
    
    return champ

def best_parent_ever(population):
    champ = max(population, key=lambda i: fitness(i))
    return champ

### Survival Selection

In [21]:
def survival_selection(population):
    population.sort(key=lambda i:fitness(i), reverse=True)       # ORDERING FROM BEST TO WORSE
    return population[:POPULATION_SIZE]       # SURVIVAL SELECTION
    
def remove_twin(population):        # Remove TWIN from the population because I belive that they will have the same fitness
    twins = []
    for i in range(len(population)):
        for j in range(i, len(population)):
            if population[i]==population[j]:
                twins.append(j)
    
    new_p = [ind for i, ind in enumerate(population) if i not in twins]
    return new_p



### Island Definition

## Problem start from HERE ->

In [22]:
starting_population = [choices([0, 1], k=NUM_LOCI) for _ in range(POPULATION_SIZE)]

In [23]:
print(len(starting_population))

print(starting_population[1]==starting_population[9])

100
False


In [24]:
math.ceil(np.random.normal(loc=10, scale=5))


4

In [25]:
population = starting_population
mutation_strategy = (mutation1, mutation2)
recombination_strategy = (one_cut_crossover, uniform_crossover)
parent_selection_strategy = (roulette_wheel, static_tournament, variable_tournament)
survival_selection_strategy = (remove_twin, survival_selection)

while all(fitness(i) != 1 for i in population):
    offspring = list()
    for counter in range(NEW_OFFSPRING):
        if random() < MUTATION_PROBABILITY:
            # MUTATION 
            p = roulette_wheel(population)
            o = mutation1(p)
        else:
            #CROSS-OVER
            p1 = variable_tournament(population)
            p2 = variable_tournament(population)
            o =  one_cut_crossover(p1, p2)
        offspring.append(o)

    population.extend(offspring)
    population = survival_selection(population)
    print(fitness(population[0]))
print(fitness.calls)

ZeroDivisionError: division by zero